# AI-Enhanced Fraud Data Generation

This notebook demonstrates how to use AI to learn from existing fraud data and generate additional sophisticated fraud transactions. The AI learns patterns from the original 100,000 transactions and creates 30,000 new AI-enhanced transactions.

## AI Enhancement Process

```mermaid
graph TD
    A[Existing 100K Transactions] --> B[AI Pattern Learning]
    B --> C[Fraud Pattern Analysis]
    B --> D[Amount Prediction Models]
    B --> E[Anomaly Detection Training]
    C --> F[Generate 30K AI Transactions]
    D --> F
    E --> F
    F --> G[Incremental Neptune Loading]
    
    style A fill:#e1f5fe
    style F fill:#f3e5f5
    style G fill:#e8f5e8
```

## What You'll Build
- **Pattern Learning**: AI learns from existing fraud patterns
- **Amount Prediction**: Smart transaction amount generation
- **Anomaly Detection**: Enhanced risk scoring
- **AI Confidence Metrics**: Quality assessment of generated data
- **Incremental Loading**: Add to existing Neptune data

**Prerequisites**: Run `Enhanced_Fraud_Bulk_Load_Workflow.ipynb` first to generate the base 100K transactions.

## Setup and Configuration

In [ ]:
# Load graph notebook extensions
%load_ext graph_notebook.magics

# Import required libraries
import pandas as pd
import boto3
import json
import os
from src.ai_learning_enhancer import AILearningEnhancer
from src.neptune_bulk_loader import NeptuneBulkLoader

# Auto-detect configuration
session = boto3.Session()
account_id = boto3.client('sts').get_caller_identity()['Account']
region = session.region_name

print("🤖 AI-Enhanced Fraud Data Generator - Ready!")
print(f"Account ID: {account_id}")
print(f"Region: {region}")

# Configuration
NEPTUNE_ENDPOINT = os.environ.get('NEPTUNE_ENDPOINT', 'UPDATE-ME.cluster-xyz.us-west-2.neptune.amazonaws.com')
S3_BUCKET = f"{account_id}-neptune-bulk-load"
NEPTUNE_ROLE_ARN = f"arn:aws:iam::{account_id}:role/neptune-workbench-NeptuneS3AccessRole"

print(f"\nNeptune Endpoint: {NEPTUNE_ENDPOINT}")
if 'UPDATE-ME' in NEPTUNE_ENDPOINT:
    print('⚠️  UPDATE NEPTUNE_ENDPOINT above with your actual Neptune cluster endpoint')
else:
    print('✅ Ready for AI enhancement!')

## Step 1: Load and Validate Existing Data

In [ ]:
# Initialize AI Learning Enhancer
ai_enhancer = AILearningEnhancer()

# Load existing dataset (will error if not found)
try:
    ai_enhancer.load_existing_data('enhanced_output')
    print("\n✅ Successfully loaded existing dataset!")
    print("   Ready to proceed with AI enhancement.")
except FileNotFoundError as e:
    print(f"\n❌ Error: {e}")
    print("\n🔧 Solution: Run 'Enhanced_Fraud_Bulk_Load_Workflow.ipynb' first to generate base data.")
    raise
except Exception as e:
    print(f"\n❌ Unexpected error: {e}")
    raise

## Step 2: Train AI Models on Existing Data

In [ ]:
# Train AI models on the existing 100K transactions
print("🎓 Training AI models on existing fraud patterns...")
print("This will:")
print("   • Learn fraud type distributions and patterns")
print("   • Train amount prediction models")
print("   • Build anomaly detection for risk scoring")
print("   • Analyze timing patterns for each fraud type")
print("\n⏱️  This may take 1-2 minutes...")

ai_enhancer.train_ai_models()

print("\n✅ AI training complete! Models are ready to generate enhanced transactions.")

## Step 3: Generate AI-Enhanced Transactions

In [ ]:
# Generate 30,000 AI-enhanced transactions
print("🎯 Generating 30,000 AI-enhanced transactions...")
print("📊 AI enhancements include:")
print("   • Learned fraud patterns from existing data")
print("   • Smart amount prediction based on fraud types")
print("   • Optimal timing based on historical patterns")
print("   • Enhanced risk scoring with anomaly detection")
print("   • AI confidence and pattern similarity metrics")
print("\n⏱️  This may take 2-3 minutes...")

# Generate AI-enhanced transactions
ai_transactions = ai_enhancer.generate_ai_enhanced_transactions(30000)

print("\n✅ AI-enhanced transaction generation complete!")

## Step 4: Examine AI-Enhanced Data

In [ ]:
# Convert to DataFrame for analysis
ai_df = pd.DataFrame(ai_transactions)

print(f"📈 AI-Enhanced Data Summary:")
print(f"  Total AI Transactions: {len(ai_df):,}")

# Fraud statistics
ai_fraud_df = ai_df[ai_df['is_fraud'] == True]
print(f"\n🚨 AI Fraud Statistics:")
print(f"  AI Fraud Transactions: {len(ai_fraud_df):,}")
print(f"  AI Fraud Rate: {len(ai_fraud_df)/len(ai_df)*100:.2f}%")

# Show AI fraud type distribution
print(f"\n🎭 AI Fraud Types:")
ai_fraud_counts = ai_fraud_df['fraud_type'].value_counts()
for fraud_type, count in ai_fraud_counts.items():
    print(f"  {fraud_type}: {count:,}")

# AI-specific metrics
print(f"\n🤖 AI Enhancement Metrics:")
print(f"  Average AI Confidence: {ai_df['ai_confidence'].mean():.3f}")
print(f"  Average Pattern Similarity: {ai_df['pattern_similarity'].mean():.3f}")
print(f"  Generation Method: {ai_df['generation_method'].iloc[0]}")

In [ ]:
# Show sample AI-enhanced transactions
print("💸 Sample AI-Enhanced Transactions:")
display(ai_df[['transaction_id', 'amount', 'fraud_type', 'ai_confidence', 'pattern_similarity', 'generation_method']].head())

print("\n🚨 Sample AI-Enhanced Fraud Transactions:")
display(ai_fraud_df[['transaction_id', 'amount', 'fraud_type', 'risk_score', 'ai_confidence', 'pattern_similarity']].head())

## Step 5: Save AI-Enhanced Data

In [ ]:
# Save AI-enhanced data to separate folder
print("💾 Saving AI-enhanced data...")

ai_enhancer.save_ai_enhanced_data(ai_transactions, 'ai_enhanced_output')

print("\n✅ AI-enhanced data saved successfully!")
print("   Files are ready for incremental Neptune bulk loading.")

## Step 6: Incremental Bulk Load to Neptune

In [ ]:
# Initialize bulk loader for incremental loading
bulk_loader = NeptuneBulkLoader(
    neptune_endpoint=NEPTUNE_ENDPOINT,
    s3_bucket=S3_BUCKET,
    neptune_role_arn=NEPTUNE_ROLE_ARN
)

print("🌊 Starting incremental bulk load to Neptune...")
print("This will:")
print("  1️⃣ Convert AI-enhanced data to Neptune CSV format")
print("  2️⃣ Upload to S3 bucket (separate prefix)")
print("  3️⃣ Start Neptune incremental bulk load job")
print("  4️⃣ Monitor progress until complete")
print("\n⏱️  Total time: ~2-3 minutes for incremental loading")
print("\n📝 Note: This will ADD the 30K transactions to existing 100K in Neptune")

In [ ]:
# Execute incremental bulk load
print("🚀 Starting incremental bulk load to Neptune...")

# Use RESUME mode for incremental loading
success = bulk_loader.bulk_load_enhanced_fraud_data('ai_enhanced_output')

if success:
    print("\n🎉 SUCCESS: AI-enhanced transactions added to Neptune!")
    print("\n📊 Final Data Status:")
    print("   ✅ Original: 100,000 transactions (rule-based)")
    print("   ✅ AI-Enhanced: 30,000 transactions (AI-generated)")
    print("   ✅ Total in Neptune: 130,000 transactions")
    print("\n📊 Next Steps:")
    print("   • Open 'Fraud_Detection_Analytics.ipynb' for analysis")
    print("   • Query both rule-based and AI-enhanced transactions")
    print("   • Compare AI confidence scores and patterns")
    print("   • Explore enhanced fraud detection capabilities")
else:
    print("\n💥 FAILED: Incremental bulk load unsuccessful.")
    print("Check Neptune logs and CloudFormation stack outputs.")

## Step 7: Verify Combined Dataset

In [ ]:
# Query Neptune to verify total transaction count
print("🔍 Verifying combined dataset in Neptune...")

# Simple Gremlin query to count total transactions
try:
    %gremlin
    g.E().hasLabel('PAYMENT').count()
except:
    print("Note: Run this cell manually to verify transaction count in Neptune")
    print("Expected result: 130,000 total transactions")

print("\n📊 Dataset Composition:")
print("   • Rule-based transactions: ~100,000 (generation_method not set)")
print("   • AI-enhanced transactions: ~30,000 (generation_method = 'AI_Enhanced')")
print("   • Total fraud transactions: ~3,900 (3% of 130K)")
print("   • AI confidence scores: Available for AI-enhanced transactions")
print("   • Pattern similarity: Available for AI-enhanced transactions")

## Summary

✅ **AI Enhancement Completed Successfully:**
1. Loaded and validated existing 100,000 transactions
2. Trained AI models on fraud patterns, amounts, and timing
3. Generated 30,000 AI-enhanced transactions with learned patterns
4. Added AI confidence and pattern similarity metrics
5. Incrementally loaded to Neptune (total: 130,000 transactions)

🤖 **AI Enhancements Added:**
- **Pattern Learning**: Fraud types based on historical frequency
- **Smart Amount Prediction**: Amounts based on learned distributions
- **Timing Optimization**: Hours based on fraud-type patterns
- **Anomaly Detection**: Enhanced risk scoring with ML
- **Quality Metrics**: AI confidence and pattern similarity scores

📊 **Next Steps:**
- Analyze AI vs rule-based transaction patterns
- Use AI confidence scores for fraud detection
- Compare pattern similarity across fraud types
- Build ML models on the enhanced 130K dataset